# Setting

In [1]:
import bt
import FinanceDataReader as fdr
import pandas as pd

from pf_utils import import_rate1, import_rate2, get_price
from pf_utils import backtest, get_start_dates, metrics

In [2]:
path_data = 'data'

In [3]:
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

# Data

## TDF

In [4]:
file = 'K55206C95800.csv'

# 2024.06.10 기준가:	1,291.67
# 2023.06.14 기준가:	1,154.82	
data_check = [
    ('2024-6-10', 1291.67),
    ('2023-6-14', 1154.82)
]

df = import_rate1(file, path_data)
df = get_price(df, data_check)
df

error: 0.00 %


date
2018-07-26     958.141087
2018-07-27     959.099229
2018-07-30     960.823883
2018-07-31     959.769927
2018-08-01     959.386671
                 ...     
2024-06-07    1284.771384
2024-06-10    1291.670000
2024-06-11    1290.424417
2024-06-12    1292.915583
2024-06-13    1293.011398
Name: K55206C95800, Length: 1455, dtype: float64

In [5]:
df_tdf_all = df.to_frame()
df_tdf_all.head()

,K55206C95800
date,
2018-07-26,958.141087
2018-07-27,959.099229
2018-07-30,960.823883
2018-07-31,959.769927
2018-08-01,959.386671


In [6]:
file = 'K55301BU6139.csv'

# 2024.06.12	1,464.07	
# 2023.06.14	1,267.58	
data_check = [
    ('2024-6-12', 1464.07),
    ('2023-6-14', 1267.58)
]

df = import_rate2(file, path_data)
df = get_price(df, data_check)
df

error: 0.03 %


date
2021-06-12     1324.71046
2021-06-13     1324.71046
2021-06-14    1327.492352
2021-06-15    1328.817062
2021-06-16    1326.300112
                 ...     
2024-06-08    1463.937529
2024-06-09    1463.937529
2024-06-10    1463.407645
2024-06-11    1466.057066
2024-06-12        1464.07
Name: K55301BU6139, Length: 1097, dtype: object

In [7]:
df_tdf_all = df_tdf_all.join(df)
df_tdf_all.tail()

,K55206C95800,K55301BU6139
date,,
2024-06-07,1284.771384,1463.937529
2024-06-10,1291.670000,1463.407645
2024-06-11,1290.424417,1466.057066
2024-06-12,1292.915583,1464.07
2024-06-13,1293.011398,NaN


In [8]:
tdf_names ={
    'K55206C95800': '키움키워드림TDF2030증권투자신탁 1[혼합-재간접형]C-P2e(퇴직연금)',
    'K55301BU6139': '미래에셋전략배분TDF2035혼합자산자투자신탁종류C-P2e'
}

In [9]:
commissions = {
    'K55206C95800': 0.49, # yearly
    'K55301BU6139': 0.6, # yearly
}
{tdf_names[k]:v for k,v in commissions.items()}

{'키움키워드림TDF2030증권투자신탁 1[혼합-재간접형]C-P2e(퇴직연금)': 0.49,
 '미래에셋전략배분TDF2035혼합자산자투자신탁종류C-P2e': 0.6}

In [10]:
get_start_dates(df_tdf_all)

,start date
K55206C95800,2018-07-26
K55301BU6139,2021-06-14


In [11]:
start_date = '2021-06-14'

In [12]:
df_tdf = df_tdf_all.loc[start_date:]

In [13]:
df_tdf.isna().sum()

K55206C95800    0
K55301BU6139    1
dtype: int64

In [14]:
df_tdf = df_tdf.ffill()

In [15]:
df_tdf

,K55206C95800,K55301BU6139
date,,
2021-06-14,1246.349927,1327.492352
2021-06-15,1249.415978,1328.817062
2021-06-16,1250.278305,1326.300112
2021-06-17,1249.128536,1324.975402
2021-06-18,1251.044818,1323.385749
...,...,...
2024-06-07,1284.771384,1463.937529
2024-06-10,1291.670000,1463.407645
2024-06-11,1290.424417,1466.057066


## ETF

In [16]:
file = 'etfs_selected_240611.csv'
df_etf_all = pd.read_csv(f'{path_data}/{file}', parse_dates=[0], index_col=0)
df_etf_all.head()

,273130,453850,284430,069500,379800,411060,305080,304660,148070,261220
Date,,,,,,,,,,
2002-10-14,NaN,NaN,NaN,5184,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-15,NaN,NaN,NaN,5312,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-16,NaN,NaN,NaN,5335,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-17,NaN,NaN,NaN,5410,NaN,NaN,NaN,NaN,NaN,NaN
2002-10-18,NaN,NaN,NaN,5642,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_sym = fdr.StockListing('ETF/KR') # 한국 ETF 전종목

symbols = df_etf_all.columns
etf_names = df_sym.loc[df_sym.Symbol.isin(symbols)].set_index('Symbol')['Name'].to_dict()
etf_names

{'069500': 'KODEX 200',
 '273130': 'KODEX 종합채권(AA-이상)액티브',
 '379800': 'KODEX 미국S&P500TR',
 '453850': 'ACE 미국30년국채액티브(H)',
 '148070': 'KOSEF 국고채10년',
 '304660': 'KODEX 미국30년국채울트라선물(H)',
 '411060': 'ACE KRX금현물',
 '305080': 'TIGER 미국채10년선물',
 '284430': 'KODEX 200미국채혼합',
 '261220': 'KODEX WTI원유선물(H)'}

In [18]:
#etf = ['069500', '148070', '305080', '379800']
etf = ['069500', '273130', '148070', '379800']
{x: etf_names[x] for x in etf}

{'069500': 'KODEX 200',
 '273130': 'KODEX 종합채권(AA-이상)액티브',
 '148070': 'KOSEF 국고채10년',
 '379800': 'KODEX 미국S&P500TR'}

In [19]:
df_etf = df_etf_all.loc[start_date:, etf]
get_start_dates(df_etf)

,start date
069500,2021-06-14
273130,2021-06-14
148070,2021-06-14
379800,2021-06-14


# Simulation

In [ ]:
class backtest():
    def __init__():
        self.stg_dict = dict()
        self.wts_dict = dict()

    def strategy(name, df, buy_n_hold=False):
        if buy_n_hold:
            weights = {name:1}
            stg_dict[col] = buy_and_hold(df, col)
            wts_dict[col] = weights

In [21]:
stg_dict = dict()
wts_dict = dict()

In [30]:
name = 'KOSPI'

index = df_tdf.index
df = df_etf.loc[index.min():index.max(), ['069500']]

weights = {name:1}
stg_dict[name] = backtest(df, weights, name, period=None)
wts_dict[name] = weights

In [28]:
name = 'TDF COMB'
weights = dict(zip(df_tdf.columns, [0.5, 0.5]))
stg_dict[name] = backtest(df_tdf, weights, name, period=None)
wts_dict[name] = weights

In [68]:
col = 'ETF COMB1'

weights = [0.2, 0.2, 0.3, 0.3]
weights = dict(zip(etf, weights))
print({etf_names[k]:v for k,v in weights.items()})

stg_dict[col] = backtest(df_etf2, weights, col, period='Y')
wts_dict[col] = weights

{'KODEX 200': 0.2, 'KODEX 종합채권(AA-이상)액티브': 0.2, 'KOSEF 국고채10년': 0.3, 'KODEX 미국S&P500TR': 0.3}


In [69]:
col = 'ETF COMB2'

weights = [0.3, 0.2, 0.3, 0.2]
weights = dict(zip(etf, weights))
print({etf_names[k]:v for k,v in weights.items()})

stg_dict[col] = backtest(df_etf2, weights, col, period='Y')
wts_dict[col] = weights

{'KODEX 200': 0.3, 'KODEX 종합채권(AA-이상)액티브': 0.2, 'KOSEF 국고채10년': 0.3, 'KODEX 미국S&P500TR': 0.2}


In [31]:
results = bt.run(*stg_dict.values())
results.plot(freq='d');
results.stats.loc[metrics]

Exception: Cannot allocate capital to KOSPI because price is 0 as of 2021-06-14 00:00:00

In [71]:
wts_dict

{'KOSPI': {'069500': 1},
 'TDF COMB': {'K55206C95800': 0.5, 'K55301BU6139': 0.5},
 'ETF COMB1': {'069500': 0.2, '273130': 0.2, '148070': 0.3, '379800': 0.3},
 'ETF COMB2': {'069500': 0.3, '273130': 0.2, '148070': 0.3, '379800': 0.2}}

In [74]:
{etf_names[k]:v for k,v in wts_dict['ETF COMB1'].items()}

{'KODEX 200': 0.2,
 'KODEX 종합채권(AA-이상)액티브': 0.2,
 'KOSEF 국고채10년': 0.3,
 'KODEX 미국S&P500TR': 0.3}